# Cross approximation in the TT format

Using the `torchtt.TT` constructor, a TT decomposition of a given tensor can be obtained. However, in the cases where the entries of the tensor are computed using a given function, building full tensors becomes unfeasible. It is possible to construct a TT decomposition using only a part of the entries of the full tensor. This is called the cross approximation method.

Imports:

In [1]:
import torch as tn
import torchtt as tntt


/Users/temporary/anaconda3/envs/torch_mkl/lib/python3.10/site-packages/torchtt/_dmrg.py:19: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/Users/temporary/anaconda3/envs/torch_mkl/lib/python3.10/site-packages/torchtt/_amen.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/Users/temporary/anaconda3/envs/torch_mkl/lib/python3.10/site-packages/torchtt/solvers.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/Users/temporary/anaconda3/envs/torch_mkl/lib/python3.10/site-packages/torchtt/cpp.py:12: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/Users/temporary/anaconda3/envs/torch_mkl/lib/python3.10/site-packages/torchtt/__init__.py:34: UserWarning: 
C++ implementation not available. Using pure Pyth

### Cross interpolation of a tensor in TT format

We want to approximate the tensor $\mathsf{x}_{i_1...i_d}=\frac{1}{2+i_1+\cdots+i_d}$. Since the passed indices are integers of type `torch.int64`, casting is used.

In [2]:
func1 = lambda I: 1/(2+tn.sum(I+1,1).to(dtype=tn.float64))

Call the `torchtt.interpolate.dmrg_cross()` method.

In [3]:
N = [20]*4
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-7)

Compute the full tensor and compare to the reference.

In [4]:
Is = tntt.meshgrid([tn.arange(0,n,dtype=tn.float64) for n in N])
x_ref = 1/(2+Is[0].full()+Is[1].full()+Is[2].full()+Is[3].full()+4)
print('Relative error ',tn.linalg.norm(x.full()-x_ref)/tn.linalg.norm(x_ref))

Relative error  tensor(6.0585e-09, dtype=torch.float64)


We consider the case $d=10$, $n_i=32$. the full tensor would contain $32^{10}$ entries.
The total number of functions calls is in this case 25000000 compared to $32^{10}$ of the total number of entries.

In [5]:
N = [32]*10
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-10, verbose=True)

Sweep 1: 
	LR supercore 1,2
		number evaluations 2048
		rank updated 2 -> 8, local error 8.166608e+05
	LR supercore 2,3
		number evaluations 16384
		rank updated 2 -> 9, local error 1.178700e+00
	LR supercore 3,4
		number evaluations 18432
		rank updated 2 -> 9, local error 1.067508e+00
	LR supercore 4,5
		number evaluations 18432
		rank updated 2 -> 9, local error 1.114764e+00
	LR supercore 5,6
		number evaluations 18432
		rank updated 2 -> 10, local error 9.758929e-01
	LR supercore 6,7
		number evaluations 20480
		rank updated 2 -> 11, local error 1.040452e+00
	LR supercore 7,8
		number evaluations 22528
		rank updated 2 -> 14, local error 1.077660e+00
	LR supercore 8,9
		number evaluations 28672
		rank updated 2 -> 13, local error 1.221364e+00
	LR supercore 9,10
		number evaluations 13312
		rank updated 2 -> 12, local error 1.212691e+00
	RL supercore 9,10
		number evaluations 13312
		rank updated 12 -> 12, local error 2.037561e-12
	RL supercore 8,9
		number evaluations 172032
		rank

The adaptive cross method used only a fraction of function calls from the original tensor.
Check some entries (full tensor cannot be computed this time) and show the rank and the storage requirements.

In [6]:
print(x[1,2,3,4,5,6,7,8,9,11], ' reference ', func1(tn.tensor([[1,2,3,4,5,6,7,8,9,11]])))
print(x[12,23,17,25,30,0,7,8,9,11], ' reference ', func1(tn.tensor([[12,23,17,25,30,0,7,8,9,11]])))
print(x)

tensor(0.0147, dtype=torch.float64)  reference  tensor([0.0147], dtype=torch.float64)
tensor(0.0065, dtype=torch.float64)  reference  tensor([0.0065], dtype=torch.float64)
TT with sizes and ranks:
N = [32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
R = [1, 13, 14, 14, 14, 14, 14, 13, 12, 11, 1]

Device: cpu, dtype: torch.float64
#entries 46720 compression 4.149569576838985e-11



### Element wise application of an univariate function on a TT tensor.

Let $f:\mathbb{R}\rightarrow\mathbb{R}$ be a function and $\mathsf{x}\in\mathbb{R}^{N_1\times\cdots\times N_d}$ be a tensor with a known TT approximation. The goal is to determine the TT approximation of $\mathsf{y}_{i_1...i_d}=f(\mathsf{x}_{i_1...i_d})$ within a prescribed relative accuracy $\epsilon$ (passed as argument).
In this case the function is `torchtt.interpoalte.function_interpolate()` and takes as arguments a function handle, the tensor $\mathsf{x}$, the accuracy epsilon, a initial tensor (starting point), number of sweeps (nswp) and the size of the rank enrichment (kick).
Further arguments are the dtype of the result and the verbose flag.

The function handle as argument gets as arguments torch vectors and has to return torch vectors of the same size.

The following example computes the elemntwise natural logarithm of a tensor. The relative error of the result is also reported.


In [7]:
x = tntt.TT(x_ref)
func = lambda t: tn.log(t)
y = tntt.interpolate.function_interpolate(func, x, 1e-9)
print('Relative error ',tn.linalg.norm(y.full()-func(x_ref))/tn.linalg.norm(func(x_ref)))

Relative error  tensor(2.6403e-11, dtype=torch.float64)


### Element wise application of an multivariate function on a TT tensor.

Let $f:\mathbb{R}\rightarrow\mathbb{R}$ be a function and $\mathsf{x}^{(1)},...,\mathsf{x}^{(d)}\in\mathbb{R}^{N_1\times\cdots\times N_d}$ be tensors with a known TT approximation. The goal is to determine the TT approximation of $\mathsf{y}_{i_1...i_d}=f(\mathsf{x}_{i_1...i_d}^{(1)},...,\mathsf{x}^{(d)}_{i_1...i_d})$ within a prescribed relative accuracy $\epsilon$ (passed as argument). The function is the same as in the previous case `tochtt.interpoalte.function_interpolate()`, but the second argument in this case is a list of torchtt.TT tensors. The function handle takes as argument a $M\times d$ torch.tensor and every of the $M$ lines corresponds to an evaluation of the function $f$ at a certain tensor entry. The function handle returns a torch tensor of length $M$.

The following example computes the same tensor as in the previous case, but with the `tochtt.interpoalte.function_interpolate()` method.

In [8]:
z = tntt.interpolate.function_interpolate(func1, Is)
print('Relative error ',tn.linalg.norm(z.full()-x_ref)/tn.linalg.norm(x_ref))

Relative error  tensor(3.9713e-11, dtype=torch.float64)
